## **Quick Note**

This is a 3D segmentation competition. However, I thought it might be useful to have a 2D Coco Dataset so here it is.

# **Train Notebook**

https://www.kaggle.com/code/vexxingbanana/gi-tract-2d-approach-mmdetection-training

# **References**

https://www.kaggle.com/code/vexxingbanana/sartorius-coco-dataset-notebook

https://www.kaggle.com/code/coldfir3/efficient-coco-dataset-generator

# **Install Libraries**

In [ ]:
!pip install pycocotools

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json
import tqdm
from tqdm import tqdm
import glob
import pycocotools
from pycocotools import mask
import random
import cv2
import re

# **Helper Functions**

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def load_json_to_dict(json_path):
    """ tbd """
    with open(json_path) as json_file:
        data = json.load(json_file)
    return data

def get_img_and_mask(img_path, annotation, width, height):
    """ Capture the relevant image array as well as the image mask """
    img_mask = np.zeros((height, width), dtype=np.uint8)
    for i, annot in enumerate(annotation): 
        img_mask = np.where(rle_decode(annot, (height, width))!=0, i, img_mask)
    img = cv2.imread(img_path)[..., ::-1]
    return img[..., 0], img_mask

def plot_img_and_mask(img, mask, invert_img=True, boost_contrast=True):
    """ Function to take an image and the corresponding mask and plot
    
    Args:
        img (np.arr): 1 channel np arr representing the image of cellular structures
        mask (np.arr): 1 channel np arr representing the instance masks (incrementing by one)
        invert_img (bool, optional): Whether or not to invert the base image
        boost_contrast (bool, optional): Whether or not to boost contrast of the base image
        
    Returns:
        None; Plots the two arrays and overlays them to create a merged image
    """
    plt.figure(figsize=(20,10))
    
    plt.subplot(1,3,1)
    _img = np.tile(np.expand_dims(img, axis=-1), 3)
    
    # Flip black-->white ... white-->black
    if invert_img:
        _img = _img.max()-_img
        
    if boost_contrast:
        _img = np.asarray(ImageEnhance.Contrast(Image.fromarray(_img)).enhance(16))
        
    plt.imshow(_img)
    plt.axis(False)
    plt.title("Cell Image", fontweight="bold")
    
    plt.subplot(1,3,2)
    _mask = np.zeros_like(_img)
    _mask[..., 0] = mask
    plt.imshow(mask, cmap='rainbow')
    plt.axis(False)
    plt.title("Instance Segmentation Mask", fontweight="bold")
    
    merged = cv2.addWeighted(_img, 0.75, np.clip(_mask, 0, 1)*255, 0.25, 0.0,)
    plt.subplot(1,3,3)
    plt.imshow(merged)
    plt.axis(False)
    plt.title("Cell Image w/ Instance Segmentation Mask Overlay", fontweight="bold")
    
    plt.tight_layout()
    plt.show()
    
def polygonFromMask(maskedArr, idx):
  # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
    contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    segmentation = []
    valid_poly = 0
    for contour in contours:
  # Valid polygons have >= 6 coordinates (3 points)
        if contour.size >= 6:
            segmentation.append(contour.astype(float).flatten().tolist())
            valid_poly += 1
    if valid_poly == 0:
        raise ValueError(idx)
    return [segmentation]

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

# **Data Processing**

In [ ]:
DATA_PATH = "../input/uw-madison-gi-tract-image-segmentation/"

In [ ]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")

In [ ]:
img_paths = []
img_heights = []
img_widths = []
pixel_scaling_heights = []
pixel_scaling_widths = []

for i, row in tqdm(train_df.iterrows()):
    ids = row.id.split("_")
    img_path = f"{DATA_PATH}/train/{ids[0]}/{ids[0]}_{ids[1]}/scans/{ids[2]}_{ids[3]}*"
    img = glob.glob(img_path)[0]
    
    splits = img.split(f"{ids[3]}_")[1].split("_")
    img_height = int(splits[0])
    img_width = int(splits[1])
    pixel_scaling_height = float(splits[2])
    pixel_scaling_width = float(splits[3].split(".png")[0])
    
    img_paths.append(img)
    img_heights.append(img_height)
    img_widths.append(img_width)
    pixel_scaling_heights.append(pixel_scaling_height)
    pixel_scaling_widths.append(pixel_scaling_width)

In [ ]:
train_df['img_path'] = img_paths
train_df['img_height'] = img_heights
train_df['img_width'] = img_widths
train_df['pixel_scaling_height'] = pixel_scaling_heights
train_df['pixel_scaling_width'] = pixel_scaling_widths

# **5 Fold Coco Dataset**

In [ ]:
FOLD = 1

In [ ]:
train_ids = train_df.id.unique()

In [ ]:
train_idxs = [i for i in range(len(train_ids)) if i%5!=FOLD]
val_idxs = [i for i in range(len(train_ids)) if i%5==FOLD]

In [ ]:
val_ids = train_ids[val_idxs]
train_ids = train_ids[train_idxs]

In [ ]:
categories = {"large_bowel": 1, "small_bowel": 2, "stomach": 3}

In [ ]:
output_json_dict = {
    "images": [],
    "annotations": [],
    "categories": []
}

category_dict = {"id": 1, "name": "large_bowel", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
category_dict = {"id": 2, "name": "small_bowel", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
category_dict = {"id": 3, "name": "stomach", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)

In [ ]:
def get_image_and_annot_info(df, fold, ids):
    i = 1
    g = 1
    
    for id_ in tqdm(ids):
        id_df = df[df['id'] == id_]
        
        image_id = i
        i += 1
        width = id_df.iloc[0, 5]
        height = id_df.iloc[0, 4]
        file_path = id_df.iloc[0, 3]
        
        image_info = {
            "id": image_id,
            "width": width,
            "height": height,
            "file_name": file_path
        }
        output_json_dict['images'].append(image_info)
        
        for j, row in id_df.iterrows():
            if pd.isna(row.segmentation) == False:
                annot = rle_decode(row.segmentation, (height, width))
                _, count = np.unique(annot, return_counts=True)
                annot_mask = annot.astype(np.bool)
                annot_mask = np.asfortranarray(annot_mask)
                
                Rs = mask.encode(annot_mask)
                Rs['counts'] = Rs['counts'].decode('utf-8')
                bbox = mask.toBbox(Rs)
                bbox_list = []
                for element in bbox:
                    bbox_list.append(int(element))
                    
                category = categories[row['class']]
                
                annot_dict = {
                    "category_id": category,
                    "segmentation": Rs,
                    "area": int(mask.area(Rs)),
                    "bbox": bbox_list,
                    "id": g,
                    "image_id": i,
                    "iscrowd": 0}
                
                g += 1
                output_json_dict['annotations'].append(annot_dict)

In [ ]:
get_image_and_annot_info(train_df, FOLD, train_ids)
with open(f'train_dataset_fold_{FOLD}.json', 'w') as f:
    output_json = json.dumps(output_json_dict, default=np_encoder)
    f.write(output_json)

In [ ]:
output_json_dict = {
    "images": [],
    "annotations": [],
    "categories": []
}

category_dict = {"id": 1, "name": "large_bowel", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
category_dict = {"id": 2, "name": "small_bowel", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
category_dict = {"id": 3, "name": "stomach", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)

In [ ]:
get_image_and_annot_info(train_df, FOLD, val_ids)
with open(f'val_dataset_fold_{FOLD}.json', 'w') as f:
    output_json = json.dumps(output_json_dict, default=np_encoder)
    f.write(output_json)

# **Verify Coco Json Files**

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

In [ ]:
annFile = Path(f'./train_dataset_fold_{FOLD}.json')
coco = COCO(annFile)
imgIds = coco.getImgIds()

In [ ]:
imgs = coco.loadImgs(imgIds[65:69])
_,axs = plt.subplots(len(imgs),2,figsize=(40,15 * len(imgs)))
for img, ax in zip(imgs, axs):
#     I = Image.open(img['file_name'])
    I = Image.fromarray(np.array(Image.open(img['file_name'])).astype("uint16"))
    annIds = coco.getAnnIds(imgIds=[img['id']])
    anns = coco.loadAnns(annIds)
    ax[0].imshow(I)
    ax[1].imshow(I)
    plt.sca(ax[1])
    coco.showAnns(anns, draw_bbox=True)

An upvote would be appreciated! :)